In [1]:
from __future__ import print_function
from collections import namedtuple
import cv2
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
from torch.optim.lr_scheduler import StepLR

Arg = namedtuple('Arg', ["model", # type of model: CNN or ANN
                         "batch_size", # size of training batch: 64, 128
                         "test_batch_size", # size of testing batch: 1000, 2000
                         "epochs", # how many epochs?
                         "lr", # learning rate?
                         "gamma", # if use adagrad, gamma = ?
                         "seed", # seed for random
                         "log_interval", 
                         "no_cuda", # use GPU or not: True or False
                         "dry_run", # run training 1 batch (to check error)? True or False
                         "save_model", # want to store model?
                         "checkpoint", # checkpoint path to store
                         "num_class", # output classes? 10 or 26
                         "restore_ck"]) # want to restore model from checkpoint?

In [2]:
class ANN(nn.Module):
    def __init__(self, hidden_sizes, num_class):
        super(ANN, self).__init__()
        # Complete the code (in the ...)
        if len(hidden_sizes) != 2:
            raise(AttributeError)

        self.model = nn.Sequential(
            nn.Linear(28*28, hidden_sizes[0]),
            nn.ReLU(),
            nn.Linear(hidden_sizes[0], hidden_sizes[1]),
            nn.ReLU(),
            nn.Linear(hidden_sizes[1], num_class),
        )
        
    def forward(self, x):
        x = x.view(x.shape[0], -1)
        x = self.model(x)
        return x

In [3]:
class CNN(nn.Module):
    def __init__(self, num_class):
        super(CNN, self).__init__()
        self.model = nn.Sequential(
            nn.Conv2d(1, 10, kernel_size=5),
            nn.MaxPool2d(2),
            nn.BatchNorm2d(10),
            nn.Dropout2d(p=0.2),
            nn.ReLU(),
            nn.Conv2d(10, 20, kernel_size=5),
            nn.MaxPool2d(2),
            nn.BatchNorm2d(20),
            nn.Dropout2d(p=0.2),
            nn.ReLU(),
            nn.Flatten(),
            nn.Linear(320, 160),
            nn.BatchNorm1d(160),
            nn.Dropout(),
            nn.ReLU(),
            nn.Linear(160, 50),
            nn.BatchNorm1d(50),
            nn.Dropout(),
            nn.ReLU(),
            nn.Linear(50, num_class),
        )

    def forward(self, x):
        x = self.model(x)
        return x

In [4]:
class CNN1(nn.Module):
    def __init__(self, num_class):
        super(CNN1, self).__init__()
        self.model = nn.Sequential(
            nn.Conv2d(1, 10, kernel_size=5),
            nn.MaxPool2d(2),
            nn.ReLU(),
            nn.Conv2d(10, 20, kernel_size=5),
            nn.MaxPool2d(2),
            nn.ReLU(),
            nn.Flatten(),
            nn.Linear(320, 160),
            nn.ReLU(),
            nn.Linear(160, 50),
            nn.ReLU(),
            nn.Linear(50, num_class),
        )

    def forward(self, x):
        x = self.model(x)
        return x

In [5]:
class CNN2(nn.Module):
    def __init__(self, num_class):
        super(CNN2, self).__init__()
        self.model = nn.Sequential(
            nn.Conv2d(1, 10, kernel_size=5),
            nn.MaxPool2d(2),
            nn.Dropout2d(p=0.2),
            nn.ReLU(),
            nn.Conv2d(10, 20, kernel_size=5),
            nn.MaxPool2d(2),
            nn.Dropout2d(p=0.2),
            nn.ReLU(),
            nn.Flatten(),
            nn.Linear(320, 160),
            nn.Dropout(),
            nn.ReLU(),
            nn.Linear(160, 50),
            nn.Dropout(),
            nn.ReLU(),
            nn.Linear(50, num_class),
        )

    def forward(self, x):
        x = self.model(x)
        return x

In [6]:
class CNN3(nn.Module):
    def __init__(self, num_class):
        super(CNN3, self).__init__()
        # Complete the code (in the ...)
        self.model = nn.Sequential(
            nn.Conv2d(1, 10, kernel_size=5),
            nn.MaxPool2d(2),
            nn.BatchNorm2d(10),
            nn.ReLU(),
            nn.Conv2d(10, 20, kernel_size=5),
            nn.MaxPool2d(2),
            nn.BatchNorm2d(20),
            nn.ReLU(),
            nn.Flatten(),
            nn.Linear(320, 160),
            nn.BatchNorm1d(160),
            nn.ReLU(),
            nn.Linear(160, 50),
            nn.BatchNorm1d(50),
            nn.ReLU(),
            nn.Linear(50, num_class),
        )

    def forward(self, x):
        x = self.model(x)
        return x

In [7]:
def train(args, model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        #labels = torch.eye(3)[labels].to(device)
        #target = torch.eye(10)[target]
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        #loss = F.nll_loss(output, target)
        loss = F.cross_entropy(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % args.log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))
            if args.dry_run:
                break
    
def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            #test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            test_loss += F.cross_entropy(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.4f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [19]:
from collections import OrderedDict

"""
Parameters:
-----------
model: torch model (CNN, ANN)
checkpoint: a file path to saved checkpoint
"""
def load_model(model, checkpoint):
    model.load_state_dict(torch.load(checkpoint))

In [20]:
import time 

def main(args, train_loader, test_loader=None, use_cuda=False):
    # Training settings
    torch.manual_seed(args.seed)

    device = torch.device("cuda" if use_cuda else "cpu")
    if args.model == "ANN":
        # fill the size of hidden layers in ...
        model = ANN([256, 128], args.num_class).to(device)
    elif args.model == "CNN":
        model = CNN(args.num_class).to(device)
    elif args.model == "CNN1":
        model = CNN1(args.num_class).to(device)
    elif args.model == "CNN2":
        model = CNN1(args.num_class).to(device)
    elif args.model == "CNN3":
        model = CNN3(args.num_class).to(device)
    else:
        print(f"wrong model {args.model}")
        raise(AttributeError)

    if args.restore_ck:
        # if restore_ck is passed, load model from this checkpoint
        load_model(model, args.restore_ck)
    
    optimizer = optim.Adadelta(model.parameters(), lr=args.lr)
    
    scheduler = StepLR(optimizer, step_size=10, gamma=args.gamma)
    for epoch in range(1, args.epochs + 1):
        st = time.time()
        train(args, model, device, train_loader, optimizer, epoch)
        if not test_loader is None:
            test(model, device, test_loader)
        scheduler.step()
        print("Time for running epoch %i, %.2f\n" % (epoch,time.time() - st))

    if args.save_model:
        torch.save({'model_state_dict': model.state_dict()}, args.checkpoint)

In [10]:
# run this code to download MNIST dataset. If errors occur, please create a folder named data & extract MNIST.zip to it.
from six.moves import urllib
opener = urllib.request.build_opener()
opener.addheaders = [('User-agent', 'Mozilla/5.0')]
urllib.request.install_opener(opener)

def load_mnist_data(batch_size, use_cuda=False, is_train=True):
    transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))
        ])
    kwargs = {'batch_size': batch_size}
    if use_cuda:
        cuda_kwargs = {'num_workers': 1,
                       'pin_memory': True,
                       'shuffle': True}
        kwargs.update(cuda_kwargs)
    data = datasets.MNIST('./data', train=is_train, download=True, transform=transform)
    loader = torch.utils.data.DataLoader(data,**kwargs)
    return loader

## Problem 1. ANN implimentation on mnist
- Horay! I got 98% Accuracy!

In [11]:
args = Arg(model="ANN",
           batch_size = 1000,
           test_batch_size = 1000,
           epochs = 10,
           lr = 1.0,
           gamma = 0.7,
           seed = 1234,
           log_interval = 10,
           no_cuda = False,
           dry_run = False,
           save_model = True,
           checkpoint = "./mnist_ann.pt",
           num_class = 10,
           restore_ck="")
# use_cuda = not args.no_cuda and torch.cuda.is_available()
use_cuda = True
train_loader = load_mnist_data(args.batch_size, use_cuda, True)
test_loader = load_mnist_data(args.test_batch_size, use_cuda, False)
main(args, train_loader, test_loader, use_cuda)

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.306846
Train Epoch: 1 [10000/60000 (17%)]	Loss: 0.888488
Train Epoch: 1 [20000/60000 (33%)]	Loss: 0.537910
Train Epoch: 1 [30000/60000 (50%)]	Loss: 0.314381
Train Epoch: 1 [40000/60000 (67%)]	Loss: 0.294836
Train Epoch: 1 [50000/60000 (83%)]	Loss: 0.330926

Test set: Average loss: 0.2931, Accuracy: 9066/10000 (90.6600%)

Time for running epoch 1, 4.73

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.297095
Train Epoch: 2 [10000/60000 (17%)]	Loss: 0.197884
Train Epoch: 2 [20000/60000 (33%)]	Loss: 0.205713
Train Epoch: 2 [30000/60000 (50%)]	Loss: 0.190881
Train Epoch: 2 [40000/60000 (67%)]	Loss: 0.216584
Train Epoch: 2 [50000/60000 (83%)]	Loss: 0.305251

Test set: Average loss: 0.1485, Accuracy: 9553/10000 (95.5300%)

Time for running epoch 2, 4.51

Train Epoch: 3 [0/60000 (0%)]	Loss: 0.179176
Train Epoch: 3 [10000/60000 (17%)]	Loss: 0.179090
Train Epoch: 3 [20000/60000 (33%)]	Loss: 0.144916
Train Epoch: 3 [30000/60000 (50%)]	Loss: 0.132967
Train Epoch: 3 [400

## Problem 2. CNN implementation on minst
- Horay! I got about 99% accuracy!

In [12]:
args = Arg(model="CNN",
           batch_size = 1000,
           test_batch_size = 1000,
           epochs = 10,
           lr = 1.0,
           gamma = 0.7,
           seed = 1234,
           log_interval = 10,
           no_cuda = False,
           dry_run = False,
           save_model = True,
           checkpoint = "./mnist_cnn.pt",
           num_class = 10,
           restore_ck="")
# use_cuda = not args.no_cuda and torch.cuda.is_available()
use_cuda = True
train_loader = load_mnist_data(args.batch_size, use_cuda, True)
test_loader = load_mnist_data(args.test_batch_size, use_cuda, False)
main(args, train_loader, test_loader, use_cuda)

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.448444
Train Epoch: 1 [10000/60000 (17%)]	Loss: 1.328828
Train Epoch: 1 [20000/60000 (33%)]	Loss: 0.826283
Train Epoch: 1 [30000/60000 (50%)]	Loss: 0.632001
Train Epoch: 1 [40000/60000 (67%)]	Loss: 0.481766
Train Epoch: 1 [50000/60000 (83%)]	Loss: 0.401944

Test set: Average loss: 0.1326, Accuracy: 9686/10000 (96.8600%)

Time for running epoch 1, 4.41

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.345797
Train Epoch: 2 [10000/60000 (17%)]	Loss: 0.310539
Train Epoch: 2 [20000/60000 (33%)]	Loss: 0.274107
Train Epoch: 2 [30000/60000 (50%)]	Loss: 0.266564
Train Epoch: 2 [40000/60000 (67%)]	Loss: 0.277547
Train Epoch: 2 [50000/60000 (83%)]	Loss: 0.232469

Test set: Average loss: 0.0651, Accuracy: 9814/10000 (98.1400%)

Time for running epoch 2, 4.66

Train Epoch: 3 [0/60000 (0%)]	Loss: 0.210858
Train Epoch: 3 [10000/60000 (17%)]	Loss: 0.212789
Train Epoch: 3 [20000/60000 (33%)]	Loss: 0.223557
Train Epoch: 3 [30000/60000 (50%)]	Loss: 0.176408
Train Epoch: 3 [400

## Problem 3. Different settings
- I removed some layers starting from the CNN network used in problem 2
- CNN1: removed batchnorm and dropout layer, 98.93 accuracy
- CNN2: removed batchnorm layer, 98.88 accuracy
- CNN3: removed dropout layer, 99.16% accuracy

In [13]:
args = Arg(model="CNN1",
           batch_size = 1000,
           test_batch_size = 1000,
           epochs = 10,
           lr = 1.0,
           gamma = 0.7,
           seed = 1234,
           log_interval = 10,
           no_cuda = False,
           dry_run = False,
           save_model = True,
           checkpoint = "./mnist_cnn1.pt",
           num_class = 10,
           restore_ck="")
# use_cuda = not args.no_cuda and torch.cuda.is_available()
use_cuda = True
train_loader = load_mnist_data(args.batch_size, use_cuda, True)
test_loader = load_mnist_data(args.test_batch_size, use_cuda, False)
main(args, train_loader, test_loader, use_cuda)

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.305345
Train Epoch: 1 [10000/60000 (17%)]	Loss: 1.939175
Train Epoch: 1 [20000/60000 (33%)]	Loss: 0.880527
Train Epoch: 1 [30000/60000 (50%)]	Loss: 0.769024
Train Epoch: 1 [40000/60000 (67%)]	Loss: 0.410567
Train Epoch: 1 [50000/60000 (83%)]	Loss: 0.254416

Test set: Average loss: 0.2365, Accuracy: 9321/10000 (93.2100%)

Time for running epoch 1, 4.50

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.249881
Train Epoch: 2 [10000/60000 (17%)]	Loss: 0.214848
Train Epoch: 2 [20000/60000 (33%)]	Loss: 0.217658
Train Epoch: 2 [30000/60000 (50%)]	Loss: 0.145303
Train Epoch: 2 [40000/60000 (67%)]	Loss: 0.193362
Train Epoch: 2 [50000/60000 (83%)]	Loss: 0.127105

Test set: Average loss: 0.0951, Accuracy: 9709/10000 (97.0900%)

Time for running epoch 2, 4.52

Train Epoch: 3 [0/60000 (0%)]	Loss: 0.071878
Train Epoch: 3 [10000/60000 (17%)]	Loss: 0.092237
Train Epoch: 3 [20000/60000 (33%)]	Loss: 0.151721
Train Epoch: 3 [30000/60000 (50%)]	Loss: 0.103312
Train Epoch: 3 [400

In [15]:
args = Arg(model="CNN2",
           batch_size = 1000,
           test_batch_size = 1000,
           epochs = 10,
           lr = 1.0,
           gamma = 0.7,
           seed = 1234,
           log_interval = 10,
           no_cuda = False,
           dry_run = False,
           save_model = True,
           checkpoint = "./mnist_cnn2.pt",
           num_class = 10,
           restore_ck="")
# use_cuda = not args.no_cuda and torch.cuda.is_available()
use_cuda = True
train_loader = load_mnist_data(args.batch_size, use_cuda, True)
test_loader = load_mnist_data(args.test_batch_size, use_cuda, False)
main(args, train_loader, test_loader, use_cuda)

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.305345
Train Epoch: 1 [10000/60000 (17%)]	Loss: 1.950418
Train Epoch: 1 [20000/60000 (33%)]	Loss: 0.922732
Train Epoch: 1 [30000/60000 (50%)]	Loss: 0.474127
Train Epoch: 1 [40000/60000 (67%)]	Loss: 0.449390
Train Epoch: 1 [50000/60000 (83%)]	Loss: 0.255070

Test set: Average loss: 0.2274, Accuracy: 9310/10000 (93.1000%)

Time for running epoch 1, 4.70

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.239078
Train Epoch: 2 [10000/60000 (17%)]	Loss: 0.224721
Train Epoch: 2 [20000/60000 (33%)]	Loss: 0.153609
Train Epoch: 2 [30000/60000 (50%)]	Loss: 0.449255
Train Epoch: 2 [40000/60000 (67%)]	Loss: 0.119101
Train Epoch: 2 [50000/60000 (83%)]	Loss: 0.215516

Test set: Average loss: 0.0977, Accuracy: 9681/10000 (96.8100%)

Time for running epoch 2, 4.66

Train Epoch: 3 [0/60000 (0%)]	Loss: 0.075872
Train Epoch: 3 [10000/60000 (17%)]	Loss: 0.103219
Train Epoch: 3 [20000/60000 (33%)]	Loss: 0.151234
Train Epoch: 3 [30000/60000 (50%)]	Loss: 0.104841
Train Epoch: 3 [400

In [16]:
args = Arg(model="CNN3",
           batch_size = 1000,
           test_batch_size = 1000,
           epochs = 10,
           lr = 1.0,
           gamma = 0.7,
           seed = 1234,
           log_interval = 10,
           no_cuda = False,
           dry_run = False,
           save_model = True,
           checkpoint = "./mnist_cnn3.pt",
           num_class = 10,
           restore_ck="")
# use_cuda = not args.no_cuda and torch.cuda.is_available()
use_cuda = True
train_loader = load_mnist_data(args.batch_size, use_cuda, True)
test_loader = load_mnist_data(args.test_batch_size, use_cuda, False)
main(args, train_loader, test_loader, use_cuda)

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.353957
Train Epoch: 1 [10000/60000 (17%)]	Loss: 0.607449
Train Epoch: 1 [20000/60000 (33%)]	Loss: 0.305973
Train Epoch: 1 [30000/60000 (50%)]	Loss: 0.182657
Train Epoch: 1 [40000/60000 (67%)]	Loss: 0.123905
Train Epoch: 1 [50000/60000 (83%)]	Loss: 0.096999

Test set: Average loss: 0.0894, Accuracy: 9850/10000 (98.5000%)

Time for running epoch 1, 4.73

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.082156
Train Epoch: 2 [10000/60000 (17%)]	Loss: 0.067207
Train Epoch: 2 [20000/60000 (33%)]	Loss: 0.052405
Train Epoch: 2 [30000/60000 (50%)]	Loss: 0.045633
Train Epoch: 2 [40000/60000 (67%)]	Loss: 0.053053
Train Epoch: 2 [50000/60000 (83%)]	Loss: 0.041609

Test set: Average loss: 0.0445, Accuracy: 9880/10000 (98.8000%)

Time for running epoch 2, 4.43

Train Epoch: 3 [0/60000 (0%)]	Loss: 0.024810
Train Epoch: 3 [10000/60000 (17%)]	Loss: 0.027479
Train Epoch: 3 [20000/60000 (33%)]	Loss: 0.057141
Train Epoch: 3 [30000/60000 (50%)]	Loss: 0.049674
Train Epoch: 3 [400

## Problem 4. EMnist
- This time, I added both dropout and batchnorm layers
- Horay! I got 83.98% accuracy!

In [17]:
# create a data folder & unzip emnist dataset to it. Then, you should see two folder under data (emnist_balance, emnist_test)
args = Arg(model="CNN",
           batch_size = 3000,
           test_batch_size = 1000,
           epochs = 30,
           lr = 1.0,
           gamma = 0.7,
           seed = 1234,
           log_interval = 10,
           no_cuda = False,
           dry_run = False,
           save_model = True,
           num_class = 26,
           checkpoint = "./emnist.pt",
           restore_ck="")

def loader(path):
    img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    return img

train_data = ImageFolder(root="./data/emnist_balance", transform=transforms.ToTensor(), loader=loader)
test_data = ImageFolder(root="./data/emnist_test", transform=transforms.ToTensor(), loader=loader)
train_loader = DataLoader(train_data, batch_size=args.batch_size, shuffle=True, num_workers=2)
test_loader = DataLoader(test_data, batch_size=args.test_batch_size, shuffle=True, num_workers=2)
main(args, train_loader, test_loader, use_cuda=True)

Train Epoch: 1 [0/156000 (0%)]	Loss: 3.439113
Train Epoch: 1 [30000/156000 (19%)]	Loss: 2.787060
Train Epoch: 1 [60000/156000 (38%)]	Loss: 2.309239
Train Epoch: 1 [90000/156000 (58%)]	Loss: 1.936414
Train Epoch: 1 [120000/156000 (77%)]	Loss: 1.711180
Train Epoch: 1 [150000/156000 (96%)]	Loss: 1.510648

Test set: Average loss: 1.7689, Accuracy: 13059/26000 (50.2269%)

Time for running epoch 1, 3.62

Train Epoch: 2 [0/156000 (0%)]	Loss: 1.485983
Train Epoch: 2 [30000/156000 (19%)]	Loss: 1.345832
Train Epoch: 2 [60000/156000 (38%)]	Loss: 1.260921
Train Epoch: 2 [90000/156000 (58%)]	Loss: 1.191450
Train Epoch: 2 [120000/156000 (77%)]	Loss: 1.106349
Train Epoch: 2 [150000/156000 (96%)]	Loss: 1.063372

Test set: Average loss: 1.0512, Accuracy: 17988/26000 (69.1846%)

Time for running epoch 2, 3.65

Train Epoch: 3 [0/156000 (0%)]	Loss: 1.035487
Train Epoch: 3 [30000/156000 (19%)]	Loss: 0.981803
Train Epoch: 3 [60000/156000 (38%)]	Loss: 0.918141
Train Epoch: 3 [90000/156000 (58%)]	Loss: 0.9278

## Problem 5. Transfer learning
- Mnist -> Eminst using checkpoint from the problem 2
- I don't think it's proper to use network that trained using easier problem

In [22]:
args = Arg(model="CNN",
           batch_size = 3000,
           test_batch_size = 1000,
           epochs = 5,
           lr = 1.0,
           gamma = 0.7,
           seed = 1234,
           log_interval = 10,
           no_cuda = False,
           dry_run = False,
           save_model = True,
           num_class = 26,
           checkpoint = "./emnist_transfer_from_mnist_cnn.pt",
           restore_ck="./mnist_cnn.pt")

def loader(path):
    img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    return img

train_data = ImageFolder(root="./data/emnist_balance", transform=transforms.ToTensor(), loader=loader)
test_data = ImageFolder(root="./data/emnist_test", transform=transforms.ToTensor(), loader=loader)
train_loader = DataLoader(train_data, batch_size=args.batch_size, shuffle=True, num_workers=2)
test_loader = DataLoader(test_data, batch_size=args.test_batch_size, shuffle=True, num_workers=2)
main(args, train_loader, test_loader, use_cuda=True)

RuntimeError: Error(s) in loading state_dict for CNN:
	Missing key(s) in state_dict: "model.0.weight", "model.0.bias", "model.2.weight", "model.2.bias", "model.2.running_mean", "model.2.running_var", "model.5.weight", "model.5.bias", "model.7.weight", "model.7.bias", "model.7.running_mean", "model.7.running_var", "model.11.weight", "model.11.bias", "model.12.weight", "model.12.bias", "model.12.running_mean", "model.12.running_var", "model.15.weight", "model.15.bias", "model.16.weight", "model.16.bias", "model.16.running_mean", "model.16.running_var", "model.19.weight", "model.19.bias". 
	Unexpected key(s) in state_dict: "model_state_dict". 